In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from datetime import datetime, timedelta

In [3]:
# 셀레니움으로 url 주소 크롬 열기
driver = webdriver.Chrome()
url = 'https://www.kobis.or.kr/kobis/business/stat/boxs/findPeriodBoxOfficeList.do'
driver.get(url)
time.sleep(1)

# 초기 날짜 지정
start_day = datetime.strptime("2017-01-01", '%Y-%m-%d')

while True:
    end_day = start_day + timedelta(days=6)
    
    start_day_str = start_day.strftime('%Y-%m-%d')
    
    end_day_str = end_day.strftime('%Y-%m-%d')
    
    # 조회기간 시작일 비우기
    driver.find_element(By.NAME,'sSearchFrom').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 마지막일 비우기
    driver.find_element(By.NAME,'sSearchTo').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 시작일 값 입력
    driver.find_element(By.NAME,'sSearchFrom').send_keys(start_day_str)
    time.sleep(0.5)
    driver.find_element(By.XPATH,'/html/body').click()
    time.sleep(0.5)
    
    # 자동으로 채워진 마지막일 값 비우기
    driver.find_element(By.NAME,'sSearchTo').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 마지막일 값 입력
    driver.find_element(By.NAME,'sSearchTo').send_keys(end_day_str)
    time.sleep(0.5)
    driver.find_element(By.XPATH,'/html/body').click()
    time.sleep(0.5)
    
    # 조회 버튼 클릭
    driver.find_element(By.CSS_SELECTOR,'div.wrap_btn .btn_blue').click()
    time.sleep(2)

    # 엑셀 파일 다운로드
    driver.find_element(By.CSS_SELECTOR,'div.tar a:nth-child(2)').click()
    time.sleep(2)

    # 자바스크립트 확인
    Alert(driver).accept()
    time.sleep(3)
    
    # 시작일 변경
    start_day = end_day + timedelta(days=1)
    
    # 최종일 지정
    if start_day == datetime.strptime("2023-05-07", '%Y-%m-%d'):
        break

# 파일 합치기

In [2]:
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [3]:
file = glob(r'C:\workspace\City_Rock\기간별_일별_2017년부터_xlsx\*.xlsx')[0]
base = pd.read_excel(file, engine='openpyxl')

In [7]:
columns_list = []
columns_list.extend(base.iloc[3][:11])
columns_list.extend(base.iloc[4][11:])
columns_list

col_list = []
col_list.extend(columns_list[0:11])
col_list.extend(['기준일'])
col_list.extend(columns_list[11:17])
col_list

['순위',
 '영화명',
 '개봉일',
 '대표국적',
 '국적',
 '제작사',
 '배급사',
 '등급',
 '장르',
 '감독',
 '배우',
 '기준일',
 '매출액',
 '누적매출액',
 '관객수',
 '누적관객수',
 '스크린수',
 '상영횟수']

In [8]:
df = pd.DataFrame(columns = col_list)
df

,순위,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수


In [9]:
df = pd.DataFrame(columns = col_list)

for file in glob(r'C:\workspace\City_Rock\기간별_일별_2017년부터_xlsx\*.xlsx'):
    base = pd.read_excel(file, engine='openpyxl')

    for i in range(11,53,6):
        info = base.iloc[5:,0:11]
        sales = base.iloc[5:, i:i+6]
        date = base.iloc[:, i:i+6].iloc[3][0]
        
        info['date'] = date
        
        data = pd.concat([info, sales],axis=1)
        data.columns = col_list
        df = pd.concat([df, data])
        

In [ ]:
df1 = df[df['순위']!='합계']

In [40]:
df1.to_csv('기간별_일별(2017.01~2023.05)_수정.csv', index=False)